In [50]:
import pandas as pd 

df = pd.read_csv('data/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [51]:
# tokenize 
def tokenize(text):
    # preprocess
    text = text.lower()

    text = text.replace('?','')

    text = text.replace("'","")
    
    return text.split()

In [52]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [53]:
# vocab form
vocab = {'<UNK>':0}

In [54]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [55]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [56]:
len(vocab)

324

In [57]:
# convert words to numerical Indices

def text_to_indices(text,vocab):

    indexed_text = []

    for token in tokenize(text):

        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [58]:
text_to_indices("what is compusx",vocab)

[1, 2, 0]

In [59]:
import torch
from torch.utils.data import Dataset, DataLoader

In [60]:
class QADataset(Dataset):
    def __init__(self,df,vocab):
        self.df = df 
        self.vocab = vocab
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

        return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [61]:
dataset = QADataset(df,vocab)

In [62]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [63]:
for question,answer in dataloader:
    print(question,answer)

tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([[156]])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[ 10,  11, 189, 158, 190]]) tensor([[191]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([[116]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([[220]])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([[207]])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([[215]])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([[185]])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([[205]])
tensor([[ 1,  2,  3, 2

In [64]:
import torch.nn as nn 

In [65]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [66]:
x = nn.Embedding(324,embedding_dim=50)

In [67]:
a = x(dataset[0][0])

In [68]:
y = nn.RNN(50,64)

In [69]:
y(a)[0].shape

torch.Size([6, 64])

In [70]:
y(a)[1].shape

torch.Size([1, 64])

In [71]:
# Hidden state
y(a)[0].shape

torch.Size([6, 64])

In [72]:
# final output O6
b = y(a)[1]
b.shape

torch.Size([1, 64])

In [73]:
z = nn.Linear(64,324)

In [74]:
z(b).shape

torch.Size([1, 324])

In [75]:
class SimpleRNN(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
        self.rnn = nn.RNN(50,64,batch_first=True)
        self.fc = nn.Linear(64,vocab_size)

    def forward(self,question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

In [76]:
learning_rate = 0.001
epochs = 20

In [77]:
model = SimpleRNN(len(vocab))

In [78]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


In [79]:
# training loop

for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:

        optimizer.zero_grad()

        #forward pass
        output = model(question)

        # loss -> output shape (1,324) - answer(1)
        loss = criterion(output,answer[0]) # answer in 2d 

        # gradients
        loss.backward()

        # update
        optimizer.step()

        total_loss = total_loss + loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss:4f}")


Epoch 1 loss: 528.416865
Epoch 2 loss: 458.887832
Epoch 3 loss: 380.948489
Epoch 4 loss: 319.012093
Epoch 5 loss: 267.829051
Epoch 6 loss: 220.533165
Epoch 7 loss: 177.506426
Epoch 8 loss: 140.318518
Epoch 9 loss: 108.534260
Epoch 10 loss: 84.542956
Epoch 11 loss: 66.194602
Epoch 12 loss: 52.555061
Epoch 13 loss: 41.967864
Epoch 14 loss: 33.900930
Epoch 15 loss: 28.427214
Epoch 16 loss: 23.535119
Epoch 17 loss: 19.914677
Epoch 18 loss: 17.056543
Epoch 19 loss: 14.550070
Epoch 20 loss: 12.925958


In [87]:
def predict(model,question,threshold=0.5):

    # convert to numbers
    numerical_question = text_to_indices(question,vocab)
    

    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # sent to model
    output = model(question_tensor)

    # convert logits to probs
    probs = torch.nn.functional.softmax(output,dim=1)

    # find index of max prob
    value , index = torch.max(probs,dim=1)

    
    if value < threshold:
        print("I don't Know")
    else:
        print(list(vocab.keys())[index])


In [88]:
predict(model,"what is the largest planet in our solar system")

jupiter


In [91]:
predict(model,"what is the capital of france")

paris
